In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import scale
from ktvgl_functions import init_parameters, fit_multivariate_t, L_update, w_update, u_update, a_update, V_update, dual_update, residual_Update, update_parameters, updateADMMpenalties

In [2]:
log_rw_returns = pd.read_csv("log_rw_returns.csv")

In [21]:
winLen = 200
Xraw = log_rw_returns[0:winLen]

maxIter = 1
update_rho = False
update_eta = True
early_stopping = False


X = scale(Xraw.to_numpy())

# number of nodes
p = X.shape[1] # n_features
k = 1 # n_clusters

# number of observations
T_n = X.shape[0]      # n_observations in Frame
sigma_e = np.exp(0.1) # Student T Distribution Standard Dev. 
alpha = 2/(T_n*sigma_e)
beta = 2*np.log(sigma_e)/T_n


rho = 2    # ADMM hyperparameter. 
gamma = 10 # hyperparameter that controls the sparsity of VAR coefficients
eta = 1e-8 # hyperparameter that controls the regularization to obtain a k-component graph
#tau = 2 # Not used
#mu = 2 # Not used


# BOOK KEEPING
# residual vectors
primal_lap_residual = np.array([], dtype=float)
primal_deg_residual = np.array([], dtype=float)
dual_residual       = np.array([], dtype=float)

lagrangian = np.array([], dtype=float)
rho_seq = np.array([], dtype=float)
eta_seq = np.array([], dtype=float)

elapsed_time = np.array([], dtype=float)


mu, Sigma, nu, history = fit_multivariate_t( X, nu_init=30.0, max_iter=200, tol=1e-6, fix_nu=None, jitter=1e-6, verbose=False)
w, Aw, a, w_lagged, L_n, Phi_n, u_n, mu_vec, d, z_n, V = init_parameters(X,k)


print("nu",nu)

for iter in range(0,maxIter):


    L_nUpdate = L_update(L_n, Phi_n, rho, k)
    wUpdate = w_update(X, w, w_lagged,a,  nu, L_n, L_nUpdate,  u_n, z_n, Phi_n, V, rho, d, eta, beta)
    u_nUpdate = u_update(wUpdate,a,w_lagged,mu_vec,rho,alpha)
    # x_update()
    aUpdate = a_update( a, wUpdate, u_nUpdate, mu_vec, gamma, rho, w_lagged)
    vUpdate = V_update(wUpdate,k)
    Phi_nUpdate, mu_vecUpdate, z_nUpdate, Phi_n_res, u_n_res, z_n_res = dual_update(rho, wUpdate, L_nUpdate, Phi_n, aUpdate, w_lagged, mu_vec, u_nUpdate, d, z_n)
    primal_lap_residual, primal_deg_residual, dual_residual = residual_Update(rho, L_n, L_nUpdate, Phi_n_res, z_n_res, primal_lap_residual, primal_deg_residual, dual_residual)
    rho, eta, rho_seq, eta_seq, has_converged = updateADMMpenalties( update_rho, rho, rho_seq,L_n, k, wUpdate, update_eta, eta, eta_seq, early_stopping)
    L_n, w, u_n, a, V, Phi_n, mu_vec, z_n = update_parameters( L_nUpdate, wUpdate, u_nUpdate, aUpdate, vUpdate, Phi_nUpdate, mu_vecUpdate, z_nUpdate)

    if has_converged:
        break




L_n

nu 96.80952131922479


c:\Users\cilli\Documents\Github\k-tvgraph\ktvgl_functions.py:436: RuntimeWarning: divide by zero encountered in divide
  f_temp = wUpdate - u_nUpdate - mu_vec/rho
c:\Users\cilli\Documents\Github\k-tvgraph\ktvgl_functions.py:439: RuntimeWarning: divide by zero encountered in divide
  thr = gamma/(rho* w_lagged[idx]**2)
c:\Users\cilli\Documents\Github\k-tvgraph\ktvgl_functions.py:403: RuntimeWarning: invalid value encountered in multiply
  temp = temp*(temp>0)
c:\Users\cilli\Documents\Github\k-tvgraph\ktvgl_functions.py:440: RuntimeWarning: invalid value encountered in cast
  a[idx] = softThresh(f_temp[idx]/w_lagged[idx], thr)


array([[ 1.25745578, -0.33372729, -0.40281017, -0.20402627, -0.31689205],
       [-0.33372729,  1.30387985, -0.34815002, -0.32991074, -0.2920918 ],
       [-0.40281017, -0.34815002,  1.28438037, -0.26297944, -0.27044074],
       [-0.20402627, -0.32991074, -0.26297944,  1.04995472, -0.25303826],
       [-0.31689205, -0.2920918 , -0.27044074, -0.25303826,  1.13246285]])

array([[ 1.25745578, -0.33372729, -0.40281017, -0.20402627, -0.31689205],
       [-0.33372729,  1.30387985, -0.34815002, -0.32991074, -0.2920918 ],
       [-0.40281017, -0.34815002,  1.28438037, -0.26297944, -0.27044074],
       [-0.20402627, -0.32991074, -0.26297944,  1.04995472, -0.25303826],
       [-0.31689205, -0.2920918 , -0.27044074, -0.25303826,  1.13246285]])